In [1]:
%load_ext sql
import pandas as pd
%sql sqlite://

'Connected: None@None'

In [2]:
# For compatibility across multiple platforms
import os
IB = os.environ.get('INSTABASE_URI',None) is not None
open = ib.open if IB else open

In [3]:
# Load tables from CSV files
# Compas Score
with open('compas-scores.csv','rU') as f:
    Scores = pd.read_csv(f, index_col=0, encoding='utf-8')
%sql drop table if exists Scores;
%sql persist Scores

Done.


u'Persisted scores'

In [4]:
%%sql
select * from Scores limit 5

Done.


id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_jail_in,c_jail_out,c_case_number,c_offense_date,c_arrest_date,c_days_from_compas,c_charge_degree,c_charge_desc,is_recid,num_r_cases,r_case_number,r_charge_degree,r_days_from_arrest,r_offense_date,r_charge_desc,r_jail_in,r_jail_out,is_violent_recid,num_vr_cases,vr_case_number,vr_charge_degree,vr_offense_date,vr_charge_desc,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,type_of_assessment,decile_score.1,score_text,screening_date
1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,0,1,0,0,0,-1.0,2013-08-13 06:03:42,2013-08-14 05:41:20,13011352CF10A,2013-08-13,None,1.0,F,Aggravated Assault w/Firearm,0,None,None,O,None,None,None,None,None,0,None,None,None,None,None,Risk of Violence,1,Low,2013-08-14,Risk of Recidivism,1,Low,2013-08-14
2,michael ryan,michael,ryan,2014-12-31,Male,1985-02-06,31,25 - 45,Caucasian,0,5,0,0,0,None,None,None,None,None,None,None,O,None,-1,None,None,O,None,None,None,None,None,0,None,None,None,None,None,Risk of Violence,2,Low,2014-12-31,Risk of Recidivism,5,Medium,2014-12-31
3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,0,3,0,0,0,-1.0,2013-01-26 03:45:27,2013-02-05 05:36:53,13001275CF10A,2013-01-26,None,1.0,F,Felony Battery w/Prior Convict,1,None,13009779CF10A,F,None,2013-07-05,Felony Battery (Dom Strang),None,None,1,None,13009779CF10A,(F3),2013-07-05,Felony Battery (Dom Strang),Risk of Violence,1,Low,2013-01-27,Risk of Recidivism,3,Low,2013-01-27
4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,0,4,0,1,4,-1.0,2013-04-13 04:58:34,2013-04-14 07:02:04,13005330CF10A,2013-04-13,None,1.0,F,Possession of Cocaine,1,None,13011511MM10A,M,0.0,2013-06-16,Driving Under The Influence,2013-06-16,2013-06-16,0,None,None,None,None,None,Risk of Violence,3,Low,2013-04-14,Risk of Recidivism,4,Low,2013-04-14
5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,0,8,1,0,1,None,None,None,13000570CF10A,2013-01-12,None,1.0,F,Possession of Cannabis,0,None,None,O,None,None,None,None,None,0,None,None,None,None,None,Risk of Violence,6,Medium,2013-01-13,Risk of Recidivism,8,High,2013-01-13


In [34]:
#Research Questions 

#1)	What is the distribution of risk assessment scores across race? 
#a.	How does the distribution look for violent risk assessment scores across race?
#2)	Does the risk assessment score correlate with actual recidivism? 
#a.	Is this true for violent risk assessment scores that predict violent recidivism? 
#b.	What is the breakdown of recidivism by race? 
#3)	Are there some crimes that correspond to certain risk scores? 
#a.	Are felony crimes consistently associated with higher or lower risk scores? What about non-felonies? 
#b.	Do some crimes correspond to a higher recidivism? 

SyntaxError: invalid syntax (<ipython-input-34-3915a0e83fbe>, line 1)

In [5]:
%%sql 
Select count(distinct(c_charge_desc))
From Scores 

Done.


count(distinct(c_charge_desc))
531


In [32]:
%%sql 
-- Figure 3.1: Looking at the number of distinct charges each race has been charged with 
select count(distinct c_charge_desc) as Count_of_Charges, race 
from Scores
group by race 

Done.


Count_of_Charges,race
377,African-American
30,Asian
310,Caucasian
177,Hispanic
20,Native American
124,Other


In [84]:
%%sql 
--  Figure 3.2, creating a tables that shows percentage of each race in data set
select race, Count/(Total*1.0) as Ratio, Count
FROM (select count() as Total From Scores) Totals,
     (select race, count() as Count from Scores group by race) Counts

Done.


race,Ratio,Count
African-American,0.494428850897,5813
Asian,0.00493323126648,58
Caucasian,0.347452581441,4085
Hispanic,0.0935612826401,1100
Native American,0.00340222845964,40
Other,0.0562218252956,661


In [33]:
%%sql 
-- Number of recidivists for every score in every race 
select decile_score as Score, race, count(is_recid) as Count of 
from scores 
where is_recid = "1" and decile_score >0 -- shows us how many who scored high on decile score actually were recidivists
group by race, decile_score

-- look at low decile scores for white and black and compare to recid category 

Done.


decile_score,race,count(is_recid)
1,African-American,107
2,African-American,143
3,African-American,177
4,African-American,204
5,African-American,199
6,African-American,246
7,African-American,270
8,African-American,277
9,African-American,300
10,African-American,251


In [9]:
%%sql 
-- Figure 3.3: Table with count of recidvists who initially committed felony crimes with risk scores of 8 or higher 
select decile_score as score, race, count(is_recid) as Count_Recid, c_charge_desc
From scores
where score >= 8 and is_recid = "1" and c_charge_desc like "%felony%"
group by race, score, c_charge_desc

Done.


score,race,Count_Recid,c_charge_desc
8,African-American,1,Felony Battery
8,African-American,3,Felony Battery w/Prior Convict
8,African-American,5,Felony Driving While Lic Suspd
8,African-American,2,Felony Petit Theft
9,African-American,1,Felony Battery
9,African-American,2,Felony Battery (Dom Strang)
9,African-American,5,Felony Battery w/Prior Convict
9,African-American,1,Felony Driving While Lic Suspd
9,African-American,8,Felony Petit Theft
10,African-American,2,Felony Batt(Great Bodily Harm)


In [13]:
%%sql
--Generates Table of number of recidivists who were arrested with no charge and had high risk assessement scores
SELECT race, decile_score as score, c_charge_desc, count(is_recid) as Count_Recid
FROM Scores 
WHERE score > 7 and is_recid = "1" and c_charge_desc like "%arrest%"
GROUP BY race, decile_score

Done.


race,score,c_charge_desc,Count_Recid
African-American,8,arrest case no charge,49
African-American,9,arrest case no charge,58
African-American,10,arrest case no charge,48
Caucasian,8,arrest case no charge,15
Caucasian,9,arrest case no charge,12
Caucasian,10,arrest case no charge,7
Hispanic,8,arrest case no charge,6
Hispanic,9,arrest case no charge,4
Hispanic,10,arrest case no charge,5
Native American,9,arrest case no charge,1


In [15]:
%%sql 
-- Figure 3.6 Number of non recidivists who had high risk scores and no intial charge
SELECT race, decile_score as score, c_charge_desc, count(is_recid) as Count_Non_Recid
FROM Scores 
WHERE score > 7 and is_recid = "0" and c_charge_desc like "%arrest%"
GROUP BY race, decile_score

Done.


race,score,c_charge_desc,Count_Non_Recid
African-American,8,arrest case no charge,65
African-American,9,arrest case no charge,74
African-American,10,arrest case no charge,66
Caucasian,8,arrest case no charge,13
Caucasian,9,arrest case no charge,22
Caucasian,10,arrest case no charge,6
Hispanic,8,arrest case no charge,6
Hispanic,9,arrest case no charge,4
Hispanic,10,arrest case no charge,3
Native American,10,arrest case no charge,1


In [6]:
%%sql 
-- Tables shows individuals with low decile scores and the corresponding number violent decile scores over 5
--individuals in those groups received
Select decile_score, race, count(v_decile_score) -- change ot is violent recid
FROM Scores 
where decile_score < 5 and decile_score != -1 and v_decile_score > 5
group by race, decile_score

Done.


decile_score,race,count(v_decile_score)
2,African-American,3
3,African-American,36
4,African-American,105
2,Caucasian,2
3,Caucasian,19
4,Caucasian,63
3,Hispanic,12
4,Hispanic,12
3,Native American,1
4,Native American,1


In [20]:
%%sql 
-- Attempting to determine if number of prior charges correlates to recidivism
-- Looking at number of prior charges compared to count of those who recidivised 
SELECT race, count(priors_count), is_recid -- can take this out 
FROM Scores 
WHERE is_recid = 1 and priors_count > 0
GROUP BY race, is_recid
-- Why was it giving me the same number of cases for both columns? 

Done.


race,count(priors_count),is_recid
African-American,1835,1
Asian,10,1
Caucasian,846,1
Hispanic,193,1
Native American,11,1
Other,105,1


In [17]:
%%sql 
-- Shows if violent decile score actually correlates to violent recidivism 
-- Similar to Figure 2.3 in Project Write Up
SELECT race, v_decile_score, count(is_violent_recid)
FROM Scores 
WHERE is_violent_recid = "1"
GROUP BY race, v_decile_score

Done.


race,v_decile_score,count(is_violent_recid)
African-American,1,40
African-American,2,40
African-American,3,67
African-American,4,56
African-American,5,58
African-American,6,79
African-American,7,61
African-American,8,49
African-American,9,61
African-American,10,30


In [32]:
%%sql 
-- Following 4 cells calculate ratio of AA and Caucasians amongst those with a risk score of 1 or 10 
SELECT R_Count/(Ta_Count*1.0) as Rate_Of_AA_Low
FROM (select race, decile_score, count() as R_Count From Scores where decile_score = "1" and race like "%African%")AA_Count,
    (select race, decile_score, count() as Ta_Count From Scores where decile_score ="1")T_Count 

Done.


Rate_Of_AA_Low
0.269305393869


In [31]:
%%sql 
SELECT R_Count_H/(Ta_Count_H*1.0) as Rate_of_AA_High
FROM (select race, decile_score, count() as R_Count_H From Scores where decile_score = "10" and race like "%African%")AA_Count_High,
    (select race, decile_score, count() as Ta_Count_H From Scores where decile_score ="10")T_Count_High 

Done.


Rate_of_AA_High
0.770491803279


In [30]:
%%sql 
SELECT R_Count/(Ta_Count*1.0) as Rate_Of_C_Low
FROM (select race, decile_score, count() as R_Count From Scores where decile_score = "1" and race like "%Cauc%")C_Count,
    (select race, decile_score, count() as Ta_Count From Scores where decile_score ="1")Tc_Count 
    

Done.


Rate_Of_C_Low
0.462553356616


In [29]:
%%sql 
SELECT R_Count/(Ta_Count*1.0) as Rate_Of_C_High
FROM (select race, decile_score, count() as R_Count From Scores where decile_score = "10" and race like "%Cauc%")C_Count_H,
    (select race, decile_score, count() as Ta_Count From Scores where decile_score ="10")Tc_Count_H 
    

Done.


Rate_Of_C_High
0.149180327869


In [14]:
%%sql 
-- These next four cells calculate the ratio for AA and Caucasians among individuals
-- who earned violent recidicism scores 1 and 10 
SELECT Count_V/(T_Count_V*1.0) as Rate_Of_AA_Low_V
FROM (select race, v_decile_score, count() as Count_V From Scores where v_decile_score = "1" and race like "%African%")AA_Count_V,
    (select race, v_decile_score, count() as T_Count_V From Scores where v_decile_score ="1")Total_V

Done.


Rate_Of_AA_Low_V
0.295028282227


In [17]:
%%sql 
SELECT Count_V_H/(T_VH*1.0) as Rate_Of_AA_High_V
FROM (select race, v_decile_score, count() as Count_V_H From Scores where v_decile_score = "10" and race like "%African%")AA_Count_VH,
    (select race, v_decile_score, count() as T_VH From Scores where v_decile_score ="10")Total_VH

Done.


Rate_Of_AA_High_V
0.78


In [18]:
%%sql 
SELECT Count_V_C/(T_C_Count_V*1.0) as Rate_Of_C_Low_V 
FROM (select race, v_decile_score, count() as Count_V_C From Scores where v_decile_score = "1" and race like "%Cauc%")C_Viol_Count,
    (select race, v_decile_score, count() as T_C_Count_V From Scores where v_decile_score ="1")Total_Viol

Done.


Rate_Of_C_Low_V
0.476332241739


In [20]:
%%sql 
SELECT Count_V_C_High/(Total_Count_V_H*1.0) as Rate_Of_C_V_High
FROM (select race, v_decile_score, count() as Count_V_C_High From Scores where v_decile_score = "10" and race like "%Cauc%") C_Count_V_H,
    (select race, v_decile_score, count() as Total_Count_V_H From Scores where v_decile_score ="10")Total_C_V_High

Done.


Rate_Of_C_V_High
0.14
